# Experiment with CIFAR10
## Federated learning with vanila

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['AUTHGRAPH_VERBOSITY'] = '0'
import pickle
import time
import datetime
import random
import operator
import copy

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)
tf.random.set_seed(6292)
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
import tensorflow_hub as hub
import numpy as np
import pandas as pd

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
today = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
today

In [ ]:
labelnames = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
labelnames

In [ ]:
dataset_root = os.path.abspath(os.path.expanduser('dataset-cifar10'))
dataset_root

In [ ]:
model_root = os.path.abspath(os.path.expanduser(f'model-cifar10-vanilla-{today}'))
model_root

In [ ]:
found = pd.read_csv('round-cifar10.csv', index_col='Clients')
found.fillna(-1, inplace=True)
found = found.astype('int')
found.drop(columns=['Total'], inplace=True)
found.drop(index=['Total'], inplace=True)
found

In [ ]:
found_round = np.unique(found.values).tolist()
found_round.remove(-1)
found_round.sort(reverse=False)
found_round

In [ ]:
found_label = dict()
for fr in found_round:
    label = found_label.get('all', dict())
    label[fr] = found.columns[((-1 < found.loc[:, :]) & 
                               (found.loc[:, :] <= fr)).any()].tolist()
    found_label['all'] = label
    for c in found.index:
        label = found_label.get(c, dict())
        label[fr] = found.columns[((-1 < found.loc[c, :]) & 
                                   (found.loc[c, :] <= fr))].tolist()
        found_label[c] = label
found_label

In [ ]:
found_client = dict()
for fr in found_round:
    found_client[fr] = found.index[((-1 < found.loc[:, :]) & 
                                    (found.loc[:, :] <= fr)).any(axis=1)].tolist()
found_client

In [ ]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
target_size = (224, 224, 3)

In [ ]:
mobilenet_v2 = 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'
inception_v3 = 'https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4'
feature_extractor_model = mobilenet_v2

In [ ]:
feature_extractor_layer = hub.KerasLayer(feature_extractor_model, input_shape=target_size, trainable=True)
feature_extractor_layer

In [ ]:
model = tf.keras.Sequential([feature_extractor_layer,
                             tf.keras.layers.Dense(len(found_label['all'][0]))
                            ])
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

In [ ]:
model_path = os.path.join(model_root, f'{0}', 'global')
model.save(model_path)